# Proyecto Individual #1

> DATAFT08 - Jorge Marcelo Mendez

### Desarrollo API

Importo librerias necesarias, asi como tambien realizo la ingesta de dataframe requerida.

In [ ]:
#Importo librerías
from fastapi import FastAPI, HTTPException
import pandas as pd

In [ ]:
#Cargo el dataset en una variable
df = pd.read_csv("streaming_completo.csv")

In [ ]:
#reviso que se haya cargado correctamente:
print(df.columns)

Ahora bien debo trabajar con los CSV's de la carpeta "ratings", ya que de ello debo sacar un valor de puntuacion promedio de cada pelicula por servicio de streaming.

---

In [ ]:
# Leer archivos CSV
ratings_1_df = pd.read_csv(r"DATASETS_Henry\ratings\1.csv") 
ratings_2_df = pd.read_csv(r"DATASETS_Henry\ratings\2.csv")
ratings_3_df = pd.read_csv(r"DATASETS_Henry\ratings\3.csv")
ratings_4_df = pd.read_csv(r"DATASETS_Henry\ratings\4.csv")
ratings_5_df = pd.read_csv(r"DATASETS_Henry\ratings\5.csv")
ratings_6_df = pd.read_csv(r"DATASETS_Henry\ratings\6.csv")
ratings_7_df = pd.read_csv(r"DATASETS_Henry\ratings\7.csv")
ratings_8_df = pd.read_csv(r"DATASETS_Henry\ratings\8.csv")

# Concatenar dataframes
ratings_df = pd.concat([ratings_1_df, ratings_2_df, ratings_3_df, ratings_4_df, ratings_5_df, ratings_6_df, ratings_7_df, ratings_8_df])

In [ ]:
# Calcular promedio de rating por movieId
ratings_df = ratings_df.groupby('movieId')['rating'].mean().reset_index()
ratings_df['rating'] = ratings_df['rating'].apply(lambda x: round(x, 1))

In [ ]:
# Eliminar duplicados
ratings_df = ratings_df.drop_duplicates(subset='movieId')

In [ ]:
ratings_df

Para poder relacionar los dataframes, debo realizar 2 acciones, la primera es eliminar la columna de show_id del dataframe de "df" y luego debo cambiar el nombre de la columna "movieId" por "id" y de esta manera hacer el merge de ambos dataframes y obtener uno consolidado con el nombre de "streaming_ratings_df".

In [ ]:
df = df.drop(columns=['show_id'])

Reviso mis cambios

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [ ]:
ratings_df = ratings_df.rename(columns={'movieId': 'id'})

Reviso si los cambios fueron realizados en ambas base de datos, para luego hacer el merge y tener un solo dataframe consolidado

In [ ]:
print(ratings_df.columns)

In [ ]:
ratings_df.head()

In [ ]:
# Unir dataframes
streaming_ratings_df = pd.merge(df, ratings_df, on='id')

In [ ]:
print(streaming_ratings_df.columns)

Al revisar las columnas observamos que 'rating_x' debiera ser 'rating', 'rating_y' debiera ser 'scored' y 'release_year' simplemente,'year'

In [ ]:
streaming_ratings_df = streaming_ratings_df.rename(columns={'rating_x': 'rating'})
streaming_ratings_df = streaming_ratings_df.rename(columns={'rating_y': 'scored'})
streaming_ratings_df = streaming_ratings_df.rename(columns={'release_year': 'year'})

Verificamos si el cambio o rename de las columnas observadas realmente, se realizó

In [ ]:
print(streaming_ratings_df.columns)

In [ ]:
streaming_ratings_df

In [ ]:
#exporto el archivo a un csv nuevo:
streaming_ratings_df.to_csv("streaming_ratings.csv", index=False)

HASTA ESTA ETAPA O PUNTO todas las pruebas han sido en mi entorno local 

---

Ahora, procedemos a cumplir con cada paso requerido en esta etapa de Desarrollo API

### PRIMERA CONSULTA

Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))

In [92]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'listed_in', 'description', 'platform', 'id', 'duration_int',
       'duration_type', 'scored'],
      dtype='object')


In [93]:
import pandas as pd
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Bienvenido a mi API de streaming de películas para Henry, te saluda Jorge Marcelo Mendez"}

# Consulta 1: Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN.
@app.get("/get_max_duration")
async def get_max_duration(year: int = None, platform: str = None, duration_type: str = None):
    streaming_ratings_df = pd.read_csv('streaming_ratings.csv')
    
    # Filtrar las columnas
    filtered_df = streaming_ratings_df.loc[(streaming_ratings_df["year"] == year if year is not None else True) &
                         (streaming_ratings_df["platform"] == platform if platform is not None else True) &
                         (streaming_ratings_df["duration_type"] == duration_type if duration_type is not None else True)]

    # Verificar si el DataFrame filtrado está vacío
    if filtered_df.empty:
        return {"message": "No se encontró una película con los filtros proporcionados."}
    
    # Mostrar la película con la mayor duración
    max_duration_movie = filtered_df.loc[filtered_df["duration_int"].idxmax()]

    return max_duration_movie.to_dict()

In [94]:
streaming_ratings_df

,type,title,director,cast,country,date_added,year,rating,listed_in,description,platform,id,duration_int,duration_type,scored
0,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,documentaries,"as her father nears the end of his life, filmm...",netflix,ns1,90.0,min,3.6
1,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",netflix,ns2,2.0,season,3.6
2,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,netflix,ns3,1.0,season,3.6
3,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",netflix,ns4,1.0,season,3.6
4,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,netflix,ns5,2.0,season,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,tv show,star trek: the original series,NaN,NaN,united states,NaN,1966,tv-pg,"action, adventure, classics",the 23rd century adventures of captain james t...,hulu,hs3069,3.0,season,3.5
22994,tv show,star trek: voyager,NaN,NaN,united states,NaN,1997,tv-pg,"action, adventure, science fiction",catapulted into the distant sector of the gala...,hulu,hs3070,7.0,season,3.5
22995,tv show,the fades,NaN,NaN,united kingdom,NaN,2011,tv-14,"horror, international, science fiction",seventeen-year-old paul is haunted by apocalyp...,hulu,hs3071,1.0,season,3.5
22996,tv show,the twilight zone,NaN,NaN,united states,NaN,1959,tv-pg,"classics, science fiction, thriller",rod serling's seminal anthology series focused...,hulu,hs3072,5.0,season,3.5


Reviso con un ejemplo:

In [95]:
get_max_duration(year=2021, platform='netflix', duration_type='season')

<coroutine object get_max_duration at 0x000001B48DC42D40>

### SEGUNDA CONSULTA

Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))

Primero debo generar un archivo completo, que incluya los scores promedio

In [104]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'listed_in', 'description', 'platform', 'id', 'duration_int',
       'duration_type', 'scored'],
      dtype='object')


In [117]:
@app.get('/score_count/')
def get_score_count(platform: str, scored: float, year: int = None):
    streaming_ratings_df = pd.read_csv('streaming_ratings.csv')
    # Filtrar los datos según los valores de los parámetros
    filtered = streaming_ratings_df.copy()
    filtered = filtered[filtered['platform'] == platform]
    if year is not None:
        filtered = filtered[filtered['year'] == year]
    
    # Contar la cantidad de películas por plataforma y devolver los resultados
    count = len(filtered[filtered['scored'] > scored])
    return {'count': count}

In [118]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_score_count(platform='netflix',scored=3 , year=2009)

{'count': 152}

### TERCERA CONSULTA

Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [98]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'listed_in', 'description', 'platform', 'id', 'duration_int',
       'duration_type', 'scored'],
      dtype='object')


In [113]:
# Consulta 3: Cantidad de películas por plataforma y filtrar los resultados según la plataforma indicada
@app.get('/count_platform/')
def get_count_platform(platform: str):
    streaming_ratings_df = pd.read_csv('streaming_ratings.csv')
    count = len(streaming_ratings_df[streaming_ratings_df['platform'] == platform])
    return {'count': count}

In [114]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_count_platform(platform='netflix')

{'count': 8807}

### CUARTA CONSULTA

Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))

In [100]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'listed_in', 'description', 'platform', 'id', 'duration_int',
       'duration_type', 'scored'],
      dtype='object')


In [111]:
@app.get('/actor/')
async def get_actor(platform: str, year: int):
    streaming_ratings_df = pd.read_csv('streaming_ratings.csv')
    # Filtrar por plataforma y año
    df_filtered = streaming_ratings_df[(streaming_ratings_df['platform'] == platform) & (streaming_ratings_df['year'] == year)]
    # Concatenar los valores de la columna "cast"
    actors_df = df_filtered['cast'].str.split(', ', expand=True)
    # Contar la frecuencia de cada actor
    actor_count = actors_df.stack().value_counts().reset_index()
    # Obtener el actor con mayor frecuencia
    most_common_actor = actor_count.iloc[0]['index']
   
    return most_common_actor

In [112]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_actor(platform='netflix', year='2009')

<coroutine object get_actor at 0x000001B48DC43140>